**Model interpretation is performed using Lime**







LIME is model-agnostic, meaning that it can be applied to any machine learning model. The technique attempts to understand the model by perturbing the input of data samples and understanding how the predictions change.

LIME modifies a single data sample by tweaking the feature values and observes the resulting impact on the output.

we will use **eli5.lime.TextExplainer** to debug the prediction - to check what was important in the document to make this decision.

Create a TextExplainer instance, then pass the document to explain and a black-box classifier (a function which returns probabilities) to the fit() method, then check the explanation:

### Train XgBoost model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install contractions

     |████████████████████████████████| 245kB 6.7MB/s 
     |████████████████████████████████| 317kB 36.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=d8ed590bce734e21deb61b009e05aaf3a484fe0fd8d7657538992be4dd69ba38
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
## import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import eli5
from eli5.lime import TextExplainer

import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
train = pd.read_csv('/content/gdrive/My Drive/lily/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/lily/test.csv')

In [ ]:
output_map = {'Animal Diseases': 0,
 'Bacterial Infections and Mycoses': 3,
 'Cardiovascular Diseases': 1,
 'Chemically-Induced Disorders': 2,
 'Congenital Hereditary and Neonatal Diseases and Abnormalities': 4,
 'Digestive System Diseases': 5,
 'Disorders of Environmental Origin': 22,
 'Endocrine System Diseases': 6,
 'Eye Diseases': 7,
 'Female Urogenital Diseases and Pregnancy Complications': 8,
 'Hemic and Lymphatic Diseases': 9,
 'Immune System Diseases': 10,
 'Male Urogenital Diseases': 11,
 'Musculoskeletal Diseases': 24,
 'Neoplasms': 12,
 'Nervous System Diseases': 13,
 'Nutritional and Metabolic Diseases': 14,
 'Occupational Diseases': 15,
 'Otorhinolaryngologic Diseases': 16,
 'Parasitic Diseases': 17,
 'Pathological Conditions and Signs and Symptoms': 18,
 'Respiratory Tract Diseases': 19,
 'Skin and Connective Tissue Diseases': 25,
 'Stomatognathic Diseases': 20,
 'Virus Diseases': 21,
 'Wounds and Injuries': 23}

In [ ]:
def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.strip()  
    doc = ' '.join([w for w in doc.split() if not w in stop_words])
    norm_docs.append(doc)
  
  return norm_docs


norm_train_reviews = pre_process_corpus(list(train['abstract']))

100%|██████████| 43916/43916 [00:24<00:00, 1823.59it/s]


In [ ]:
norm_test_reviews = pre_process_corpus(list(test['abstract']))

100%|██████████| 10862/10862 [00:05<00:00, 1819.43it/s]


In [ ]:
train['categories'] = train['categories'].map(output_map)
test['categories'] = test['categories'].map(output_map)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost.sklearn import XGBClassifier

In [ ]:
tv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
xgb = XGBClassifier(learning_rate=0.1,
                    n_estimators=300,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=26,
                    seed=27)
pipe = make_pipeline(tv, xgb)
pipe.fit(norm_train_reviews, train['categories'])

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=3, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=1,
                                 stop_words='english', strip_accents='unicode',
                                 sublinear_tf=1, to...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.8, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=1, miss

### model interpretation using TextExplainer

In [ ]:
testing = norm_test_reviews[0]
testing

'reactive oxygen species ros proinflammatory cytokines pic upregulated posttraumatic stress disorder ptsd histone deacetylase inhibitors hdaci modify genetic transcription diminish ros pic escalation also modulate levels neurotransmitters catecholamines serotonin 5ht thus study sought analyze effects hdaci valproic acid va oxidative stress inflammation neurotransmitter modulation via predator exposurepsychosocial stress animal model ptsd ptsdlike effects induced male spraguedawley rats n6group4 groups rats secured plexiglas cylinders placed cage cat 1h days 1 11 40 40day stress regimen ptsd rats also subjected psychosocial stress via daily cage cohort changes conclusion stress regimen treatment group ptsdva control group controlva rats given va drinking water 30 days rats euthanized brains dissected remove hippocampus prefrontal cortex pfc whole blood collected assess systemic oxidative stress ros pic mrna protein elevation ptsd group normalized va anxiety decreased group via improved 

In [ ]:
sorted_output = {k:v for k,v in sorted(output_map.items(), key = lambda x: x[1])}

In [ ]:
te = TextExplainer(random_state=42)
te.fit(testing, pipe.predict_proba)
te.show_prediction(target_names=list(sorted_output.keys()))